In [2]:
from copy import deepcopy
from colorama import Fore, Back, Style

#direction matrix
DIRECTIONS = {"U": [-1, 0], "D": [1, 0], "L": [0, -1], "R": [0, 1]}
#target matrix
END = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# unicode for draw puzzle in command promt or terminal
left_down_angle = '\u2514'   # each one of these is a unicode for the intersection of 2 lines
right_down_angle = '\u2518'
right_up_angle = '\u2510'
left_up_angle = '\u250C'

middle_junction = '\u253C'
top_junction = '\u252C'
bottom_junction = '\u2534'
right_junction = '\u2524'
left_junction = '\u251C'

#bar color
#Style.BRIGHT make the text appear brighter
# '\u2502' unicode of vertical bar
#Fore.RESET resets the foreground color of the text back to the default color
# Style.RESET_ALL resets all the styles applied to the text back to default
bar = Style.BRIGHT + Fore.CYAN + '\u2502' + Fore.RESET + Style.RESET_ALL  
dash = '\u2500'

#Line draw code
first_line = Style.BRIGHT + Fore.CYAN + left_up_angle + dash + dash + dash + top_junction + dash + dash + dash + top_junction + dash + dash + dash + right_up_angle + Fore.RESET + Style.RESET_ALL
middle_line = Style.BRIGHT + Fore.CYAN + left_junction + dash + dash + dash + middle_junction + dash + dash + dash + middle_junction + dash + dash + dash + right_junction + Fore.RESET + Style.RESET_ALL
last_line = Style.BRIGHT + Fore.CYAN + left_down_angle + dash + dash + dash + bottom_junction + dash + dash + dash + bottom_junction + dash + dash + dash + right_down_angle + Fore.RESET + Style.RESET_ALL

#puzzle print function
def print_puzzle(array):
    print(first_line) # Print the first line of the puzzle grid
    for a in range(len(array)):  # Iterate through each row of the puzzle
        for i in array[a]: # Iterate through each element in the current row
            if i == 0:  # If the element is 0 (representing the empty space)
                # Print a vertical bar with a red background and a space (to represent the empty space)
                print(bar, Back.RED + ' ' + Back.RESET, end=' ')
            else:  # If the element is not 0
                # Print a vertical bar followed by the value of the element
                print(bar, i, end=' ')
        print(bar)  # Print the closing vertical bar for the current row
        if a == 2:  # If it's the last row of the puzzle grid
            print(last_line)  # Print the last line of the puzzle grid
        else:  # If it's not the last row
            print(middle_line)  # Print the middle line of the puzzle grid

#it is the node which store each state of puzzle
class Node:
    def __init__(self, current_node, previous_node, g, h, dir):
        # Initialize a Node object with attributes
        self.current_node = current_node  # Current state of the puzzle
        self.previous_node = previous_node # State that led to the current state
        self.g = g # Cost of reaching the current state from the initial state
        self.h = h # Heuristic value estimating the cost to reach the goal state from the current state
        self.dir = dir  # Direction taken to reach the current state

    def f(self):
        return self.g + self.h  # Calculate the total cost of the current node

# Helper function to get the position of an element within the puzzle grid
def get_pos(current_state, element):
    # Iterate through each row of the puzzle grid
    for row in range(len(current_state)):  # Check if the element is present in the current row
        if element in current_state[row]:
            return (row, current_state[row].index(element)) # If found, return the row and column indices of the element

# Definition of manhattan distance: The Manhattan distance is the sum of the absolute differences in the row and column indices
#between the current cell and its goal position.     
        
        
#it is a distance calculation algorithm
def euclidianCost(current_state):
    cost = 0
    for row in range(len(current_state)): # Iterate through each cell in the puzzle grid
        for col in range(len(current_state[0])):
            # Find the position of the current cell's value in the goal state
            pos = get_pos(END, current_state[row][col])
            # Calculate the Manhattan distance between the current cell and its goal position
            cost += abs(row - pos[0]) + abs(col - pos[1])
    return cost

#get adjucent Nodes
def getAdjNode(node): 
    listNode = []  # empty node to store adjacent nodes
    emptyPos = get_pos(node.current_node, 0)  # Get the position of the empty space (0)

     # Iterate through each direction (up, down, left, right)
    for dir in DIRECTIONS.keys():
        # Calculate the new position after moving in the current direction
        newPos = (emptyPos[0] + DIRECTIONS[dir][0], emptyPos[1] + DIRECTIONS[dir][1])
        # Check if the new position is within the bounds of the puzzle grid
        if 0 <= newPos[0] < len(node.current_node) and 0 <= newPos[1] < len(node.current_node[0]):
            # Create a new state by swapping the empty space with the adjacent tile
            newState = deepcopy(node.current_node)  # Create a deep copy of the current state
            newState[emptyPos[0]][emptyPos[1]] = node.current_node[newPos[0]][newPos[1]] # Move the tile to the empty space
            newState[newPos[0]][newPos[1]] = 0 # Set the previous tile position to empty (0)
            # Create a new Node object representing the adjacent state and add it to the list
            listNode.append(Node(newState, node.current_node, node.g + 1, euclidianCost(newState), dir))

    return listNode  # Return the list of adjacent nodes

#get the best node available among nodes
def getBestNode(openSet):
    firstIter = True

    for node in openSet.values():
        # Check if it's the first iteration or if the current node has a lower total cost (f value) than the best node so far
        if firstIter or node.f() < bestF:
            firstIter = False
            bestNode = node # Update the best node
            bestF = bestNode.f() # Update the best total cost (f value)
    return bestNode   # Return the best node

#this functionn create the smallest path
def buildPath(closedSet):
    node = closedSet[str(END)]  # Get the final node from the closed set
    branch = list()  # Initialize a list to store the path

    while node.dir:
        # Add the direction and current node to the path
        branch.append({
            'dir': node.dir,
            'node': node.current_node
        })
        # Update the current node to its previous node in the closed set
        node = closedSet[str(node.previous_node)]
        # Add the final node to the path
    branch.append({
        'dir': '',
        'node': node.current_node
    })
    # Reverse the path to get it in the correct order
    branch.reverse()

    return branch  # Return path

#main function of node
def main(puzzle):
    open_set = {str(puzzle): Node(puzzle, puzzle, 0, euclidianCost(puzzle), "")} # Initialize open set with the initial puzzle state
    closed_set = {} # Initialize an empty closed set

    while True:
        test_node = getBestNode(open_set)  # Get the best node from the open set
        closed_set[str(test_node.current_node)] = test_node  # Add the best node to the closed set

        # Check if the current node is the goal state
        if test_node.current_node == END:
            return buildPath(closed_set) # If so, return the path to the goal state

        adj_node = getAdjNode(test_node)  # Get adjacent nodes from the current node
        for node in adj_node:
            # Check if the adjacent node is already in the closed set or if it's in the open set with a lower total cost
            if str(node.current_node) in closed_set.keys() or str(node.current_node) in open_set.keys() and open_set[
                str(node.current_node)].f() < node.f():
                continue  # Skip this adjacent node
            open_set[str(node.current_node)] = node  # Add the adjacent node to the open set

        del open_set[str(test_node.current_node)]  # remove from the dictionary

        

if __name__ == '__main__':
    #it is start matrix
    br = main([[5, 3, 2],
               [4, 6, 7],
               [1,0, 8]])

    print('total steps : ', len(br) - 1)
    print()
    print(dash + dash + right_junction, "INPUT", left_junction + dash + dash)  # Print the input label
    for b in br:  # Iterate over each step in the solution path
        if b['dir'] != '':  # If the direction is not empty (not the initial state)
            letter = ''
            # Map the direction abbreviation to a human-readable string
            if b['dir'] == 'U':
                letter = 'UP'
            elif b['dir'] == 'R':
                letter = "RIGHT"
            elif b['dir'] == 'L':
                letter = 'LEFT'
            elif b['dir'] == 'D':
                letter = 'DOWN'
                # Print the direction taken for the current step
            print(dash + dash + right_junction, letter, left_junction + dash + dash)
             # Print the current puzzle state
        print_puzzle(b['node'])
        print()

    print(dash + dash + right_junction, 'ABOVE IS THE OUTPUT', left_junction + dash + dash)  # Print the output label

total steps :  23

──┤ INPUT ├──
┌───┬───┬───┐
│ 5 │ 3 │ 2 │
├───┼───┼───┤
│ 4 │ 6 │ 7 │
├───┼───┼───┤
│ 1 │   │ 8 │
└───┴───┴───┘

──┤ RIGHT ├──
┌───┬───┬───┐
│ 5 │ 3 │ 2 │
├───┼───┼───┤
│ 4 │ 6 │ 7 │
├───┼───┼───┤
│ 1 │ 8 │   │
└───┴───┴───┘

──┤ UP ├──
┌───┬───┬───┐
│ 5 │ 3 │ 2 │
├───┼───┼───┤
│ 4 │ 6 │   │
├───┼───┼───┤
│ 1 │ 8 │ 7 │
└───┴───┴───┘

──┤ LEFT ├──
┌───┬───┬───┐
│ 5 │ 3 │ 2 │
├───┼───┼───┤
│ 4 │   │ 6 │
├───┼───┼───┤
│ 1 │ 8 │ 7 │
└───┴───┴───┘

──┤ LEFT ├──
┌───┬───┬───┐
│ 5 │ 3 │ 2 │
├───┼───┼───┤
│   │ 4 │ 6 │
├───┼───┼───┤
│ 1 │ 8 │ 7 │
└───┴───┴───┘

──┤ DOWN ├──
┌───┬───┬───┐
│ 5 │ 3 │ 2 │
├───┼───┼───┤
│ 1 │ 4 │ 6 │
├───┼───┼───┤
│   │ 8 │ 7 │
└───┴───┴───┘

──┤ RIGHT ├──
┌───┬───┬───┐
│ 5 │ 3 │ 2 │
├───┼───┼───┤
│ 1 │ 4 │ 6 │
├───┼───┼───┤
│ 8 │   │ 7 │
└───┴───┴───┘

──┤ UP ├──
┌───┬───┬───┐
│ 5 │ 3 │ 2 │
├───┼───┼───┤
│ 1 │   │ 6 │
├───┼───┼───┤
│ 8 │ 4 │ 7 │
└───┴───┴───┘

──┤ UP ├──
┌───┬───┬───┐
│ 5 │   │ 2 │
├───┼───┼───┤
│ 1 │ 3 │ 6 │
├───┼───┼───┤
│ 8 │